### 5.9 含并行连结的网络（GoogLeNet）

![LeNet网络结构](img/5.9_inception.svg)

#### 5.9.1 Inception块

> 1. **有4条并行线路**
> 2. **前3条线路窗口大小分别是 1x1, 3x3, 4x4, 中间2个线路会对输入先做 1x1 卷积来减少输入通道数**
> 3. **第四条线路使用 3x3 最大池化层,后接 1x1 卷积层来改变通道数**
> 4. **4条线路都使用合适的填充是输入和输出宽高一致**

In [3]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch.utils as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class Inception(nn.Module):
    # c1 - c4 为每条线路里的层的输出通道数
    # in_c 为输入通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1x1卷积层 + 3x3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层 + 5x5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大池化层 + 1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)
    
    def forward(self, x):
        # 分别先计算四条线路的输出结果，最后将4个结果连接
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1) # 在通道维上连接输出

#### 5.9.2 GoogLeNet 模型

> 1. **和VGG一样,主体卷积部分使用5个模块,每个模块之间使用步幅为2的3x3最大池化层来见减小输出宽高**
> 2. **第一模块: 64通道的7x7卷积层, 64x(7x7)**
> 3. **第二模块: 2个卷积层,64通道的1x1,192通道的3x3卷积层, 64x(1x1) + 192x(3x3)**
> 4. **第三模块: 2个Inception块**
>     - 第一个Inception块输出通道数:64+128+32+32=256,第二、三条线路分别将输入通道数减小到96/192和16/192后,再接上第二层卷积层
>     - 第二个Inception块输出通道数:128+192+96+64=480,第二、三条线路分别将输入通道数减小到128/256和32/256
> 5. **第四模块: 串联5个Inception块**
>     - 输出通道数分别是:192+208+48+64=512, 160+224+64+64=512, 128+256+64+64=512, 112+288+64+64=528, 256+320+128+128=832
>     - 1x1卷积层在第一条线路,3x3卷积层在第二条线路,之后5x5在第三条,3x3最大池化层在第四条
> 6. **第五模块: 2个Inception块,后紧跟输出层,使用全局平均池化层来讲每个通道的宽和高变为1,将输出变为二维数组后加一个输出个数为标签类别数的全连接层**
>     - 第一个Inception块输出通道数:256+320+128+128=832
>     - 第二个Inception块输出通道数:384+384+128+128=1024

In [12]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

b2 = nn.Sequential(
    nn.Conv2d(64, 64, kernel_size=1),
    nn.Conv2d(64, 192, kernel_size=3, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

b3 = nn.Sequential(
    Inception(192, 64, (96, 128), (12, 32), 32),
    Inception(256, 128, (128, 192), (32, 96), 64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

b4 = nn.Sequential(
    Inception(480, 192, (96, 208), (16, 48), 64),
    Inception(512, 160, (112, 224), (24, 64), 64),
    Inception(512, 128, (128, 256), (24, 64), 64),
    Inception(512, 112, (144, 288), (32, 64), 64),
    Inception(528, 256, (160, 320), (32, 128), 128),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

b5 = nn.Sequential(
    Inception(832, 256, (160, 320), (32, 128), 128),
    Inception(832, 384, (192, 384), (48, 128), 128),
    d2l.GlobalAvgPool2d()
)

net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))

In [13]:
X = torch.rand(1, 1, 96, 96)

for blk in net.children():
    X = blk(X)
    print('output shape: ', X.shape)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])


#### 5.9.3 获取数据和训练模型

In [15]:
batch_size = 32

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)